In [3]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import mediapipe as mp
from tensorflow.keras.models import load_model

# Define actions and colors
actions = ['അ', 'ആ', 'ഇ' , 'ഈ' , 'ഉ', 'ഊ' , "ഋ" , 'എ' , 'ഏ' , 'ഐ' , 'ഒ' , 'ഓ', 'ഔ', 'അം', 'അഃ','ക', 'ഖ', 'ഗ' , 'ഘ' , 'ങ', 'ച' , "ഛ" , 'ജ' , 'ഝ' , 'ഞ']

colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16), (117, 245, 16), (16, 117, 245),
          (245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16), (117, 245, 16),
          (245, 117, 16), (117, 245, 16), (16, 117, 245), (245, 117, 16)]

# Load a font that supports Malayalam characters
font_path = r"C:\\Users\\amalk\\Documents\\Noto_Sans_Malayalam\\static\\NotoSansMalayalam_Condensed-Bold.ttf"
try:
    font = ImageFont.truetype(font_path, 32)
except OSError:
    print(f"Error: Cannot open the font file at {font_path}.")
    raise

# Visualization function
def prob_viz(res, actions, image, colors):
    output_image = image.copy()
    pil_image = Image.fromarray(output_image)
    draw = ImageDraw.Draw(pil_image)
    
    for idx, (prob, action, color) in enumerate(zip(res, actions, colors)):
        draw.rectangle([(0, idx * 60), (int(prob * 300), 60 + idx * 60)], fill=color)
        draw.text((10, (idx * 60) + 10), action, font=font, fill=(255, 255, 255, 255))
    
    return np.array(pil_image)

# Initialize variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Initialize video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Mediapipe model setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Load the pre-trained model
model_path = r'C:\Users\amalk\action_final.h5'
try:
    model = load_model(model_path)
    print(f"Model loaded successfully from {model_path}.")
except OSError:
    print(f"Error: Could not load model from {model_path}.")
    exit()

# Mediapipe detection function
def mediapipe_detection(image, holistic_model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Draw landmarks
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_CONTOURS)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

# Extract keypoints
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    keypoints = np.concatenate([pose, face, lh, rh])
    required_length = 1662
    if len(keypoints) < required_length:
        keypoints = np.pad(keypoints, (0, required_length - len(keypoints)), 'constant')
    return keypoints

# Main loop
def main_loop():
    global sequence, sentence, predictions
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to capture image")
                break

            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)

            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                try:
                    res = model.predict(np.expand_dims(sequence, axis=0))[0]
                    predicted_action = actions[np.argmax(res)]
                    print(f"Predicted action: {predicted_action}")

                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if predicted_action != sentence[-1]:
                                sentence.append(predicted_action)
                        else:
                            sentence.append(predicted_action)

                    if len(sentence) > 5:
                        sentence = sentence[-5:]

                    image = prob_viz(res, actions, image, colors)

                    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                    draw = ImageDraw.Draw(pil_image)
                    draw.rectangle([(0, 0), (640, 40)], fill=(245, 117, 16))
                    draw.text((10, 5), ' '.join(sentence), font=font, fill=(255, 255, 255, 255))
                    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
                except Exception as e:
                    print(f"Error during prediction: {e}")

            cv2.imshow('OpenCV Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main_loop()


Model loaded successfully from C:\Users\amalk\action_final.h5.
1/1 [==============================] - 1s 1s/step
Predicted action: ഈ
1/1 [==============================] - 0s 42ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 44ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 46ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 50ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 33ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 39ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 35ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 41ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 34ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 33ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 39ms/step
Predicted action: ഈ
1/1 [==============================] - 0s 33ms/step
Predicted action: ഈ
1/1